# Mesogens with NP | Pressure Ramps

## Pressure: 3.1 -> 3.3

### Date: 08/01/20 | System P = 3.3, Expected value of $T_c$ : 9.91

In [1]:
#-----Importa los paquetes esenciales para correr la simulación.

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables

p_prev = 3.1;
p_new = 3.3;
temp = 9.0;
steps_ramp = 1e4;
run_time = steps_ramp;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(temp) + "_P_" + str(p_prev) + "_ramp.gsd"

In [4]:
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd(ramp_file,frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#-----Integrate using NPT

pressure = hoomd.variant.linear_interp(points = [(0,p_prev), (steps_ramp, p_new)])
npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 1.1, tauP = 1.0,P = pressure);
npt.randomize_velocities(23)

#-----Define los archivos en los que se guardará la información de la simulación. 

log_file = "T_" + str(temp) + "_P_" + str(p_new) + "_ramp.log"
gsd_file = "T_" + str(temp) + "_P_" + str(p_new) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(temp) + "_P_" + str(p_new) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True);

HOOMD-blue 2.7.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/02/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1010 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [5]:
#-----Run the simulation.

hoomd.run(run_time)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9010
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 291226 / 301000 | TPS 22.5857 | ETA 00:07:12
Time 00:00:20 | Step 291457 / 301000 | TPS 22.9825 | ETA 00:06:55
Time 00:00:30 | Step 291683 / 301000 | TPS 22.5626 | ETA 00:06:52
Time 00:00:40 | Step 291903 / 301000 | TPS 21.9123 | ETA 00:06:55
Time 00:00:50 | Step 292123 / 301000 | TPS 21.9427 | ETA 00:06:44
Time 00:01:00 | Step 292346 / 301000 | TPS 22.2943 | ETA 00:06:28
Time 00:01:10 | Step 292577 / 301000 | TPS 23.0636 | ETA 00:06:05
Time 00:01:20 | Step 292813 / 301000 | TPS 23.5466 | ETA 00:05:47
Time 00:01:30 | Step 293045 / 301000 | TPS 23.0974 | ETA 00:05:44
Time 00:01:40 | Step 293269 / 301000 | TPS 22.3505 | ETA 00:05:45
Time 00:01:50 | Step 293491 / 301000 | TPS 22.1315 | ETA 00:05:39
Time 00:02:00 | Step 293709 / 3

In [6]:
#-----Get density.

meta = system.get_metadata()

In [7]:
#-----Get volume.

volume = system.box.get_volume()